In [2]:
from pathlib import Path

# base path
file_path = Path("papers 250") / "60"

md_contents = []

for folder in file_path.iterdir():
    if folder.is_dir():
        doc_folder = folder / "document"
        if doc_folder.exists():
            for md_file in doc_folder.glob("*.checked.md"):
                with md_file.open(encoding="utf-8") as f:
                    content = f.read()
                md_contents.append(content)

# Now md_contents is a list of strings
print(f"Collected {len(md_contents)} markdown files.")

Collected 60 markdown files.


In [3]:
!pip install tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.0/962.0 kB 18.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.8/287.8 kB 25.8 MB/s eta 0:00:00


In [ ]:
import tiktoken
from pathlib import Path
import pandas as pd
from openpyxl import load_workbook

# Pricing (USD per 1M tokens → per 1000 tokens)
pricing = {
    # GPT Models
    "GPT-5": {
        "input": 1.25,
        "output": 10.00,
        "cached": 0.125
    },
    "GPT-5 mini": {
        "input": 0.25,
        "output": 2.00,
        "cached": 0.025
    },
    "GPT-5 nano": {
        "input": 0.05,
        "output": 0.40,
        "cached": 0.005
    },
    "GPT-5 Chat Latest": {
        "input": 1.25,
        "output": 10.00,
        "cached": 0.125
    },
    "GPT-5 Codex": {
        "input": 1.25,
        "output": 10.00,
        "cached": 0.125
    },
    "GPT-4.1": {
        "input": 2.00,
        "output": 8.00,
        "cached": 0.50
    },
    "GPT-4.1 mini": {
        "input": 0.40,
        "output": 1.60,
        "cached": 0.10
    },
    "GPT-4.1 nano": {
        "input": 0.10,
        "output": 0.40,
        "cached": 0.025
    },
    "GPT-4o": {
        "input": 2.50,
        "output": 10.00,
        "cached": 1.25
    },

    # DeepSeek Models
    "DeepSeek Chat V3.1 (Non-thinking)": {
        "input": 0.56,
        "output": 1.68,
        "cached": 0.07
    },
    "DeepSeek Reasoner V3.1 (Thinking)": {
        "input": 0.56,
        "output": 1.68,
        "cached": 0.07
    },

    # Claude Models
    "Claude Opus 4.1": {
        "input": 15,
        "output": 75,
        "cached_5m": 18.75,
        "cached_1h": 30,
        "cache_hit": 1.50
    },
    "Claude Opus 4": {
        "input": 15,
        "output": 75,
        "cached_5m": 18.75,
        "cached_1h": 30,
        "cache_hit": 1.50
    },
    "Claude Sonnet 4": {
        "input": 3,
        "output": 15,
        "cached_5m": 3.75,
        "cached_1h": 6,
        "cache_hit": 0.30
    },
    "Claude Sonnet 3.7": {
        "input": 3,
        "output": 15,
        "cached_5m": 3.75,
        "cached_1h": 6,
        "cache_hit": 0.30
    },
    "Claude Haiku 3.5": {
        "input": 0.80,
        "output": 4,
        "cached_5m": 1,
        "cached_1h": 1.6,
        "cache_hit": 0.08
    },
    "Claude Haiku 3": {
        "input": 0.25,
        "output": 1.25,
        "cached_5m": 0.30,
        "cached_1h": 0.50,
        "cache_hit": 0.03
    }
}


# Fixed output tokens
fixed_output_tokens = 15000

# Map each model to its tiktoken encoding
model_encodings = {
    "GPT-5": "gpt-4",
    "GPT-5 mini": "gpt-4",
    "GPT-5 nano": "gpt-4",
    "GPT-5 Chat Latest": "gpt-4",
    "GPT-5 Codex": "gpt-4",
    "GPT-4.1": "gpt-4",
    "GPT-4.1 mini": "gpt-4",
    "GPT-4.1 nano": "gpt-4",
    "GPT-4o": "gpt-4",
    "DeepSeek Chat V3.1 (Non-thinking)": "gpt-4",
    "DeepSeek Reasoner V3.1 (Thinking)": "gpt-4",
    "Claude Opus 4.1": "cl100k_base",
    "Claude Opus 4": "cl100k_base",
    "Claude Sonnet 4": "cl100k_base",
    "Claude Sonnet 3.7": "cl100k_base",
    "Claude Haiku 3.5": "cl100k_base",
    "Claude Haiku 3": "cl100k_base",
}

# # Tokenizer (closest available to GPT-5 & Claude)
# encoding = tiktoken.encoding_for_model("gpt-4")

file_path = Path("papers 250") / "60"
rows = []

for folder in file_path.iterdir():
    if folder.is_dir():
        doc_folder = folder / "document"
        if doc_folder.exists():
            for md_file in doc_folder.glob("*.checked.md"):
                with md_file.open(encoding="utf-8") as f:
                    content = f.read()

                table_name = folder.name
                row = {"Table Name": table_name}

                # Compute base tokens for the document (initial input token count)
                base_enc = tiktoken.encoding_for_model("gpt-4")  # or any standard model
                base_tokens = len(base_enc.encode(content))
                row["Tokens"] = base_tokens  # store it for reference / totals

                # Now loop over models to calculate costs
                for model, rates in pricing.items():
                    encoding_name = model_encodings.get(model, "gpt-4")
                    
                    # Use correct encoding per model
                    if encoding_name.startswith("cl"):
                        enc = tiktoken.get_encoding(encoding_name)
                    else:
                        enc = tiktoken.encoding_for_model(encoding_name)
                    
                    model_tokens = len(enc.encode(content))  # optional per-model token count

                    # GPT-5 style models
                    if "cached" in rates:
                        input_cost = model_tokens * rates["input"] / 1_000_000
                        output_cost = fixed_output_tokens * rates["output"] / 1_000_000
                        cached_cost = model_tokens * rates["cached"] / 1_000_000
                        total_cost = input_cost + output_cost
                        row[f"{model} (input)"] = input_cost
                        row[f"{model} (output, 4000)"] = output_cost
                        row[f"{model} (cached)"] = cached_cost
                        row[f"{model} (total)"] = total_cost

                    # Claude style models
                    else:
                        input_cost = model_tokens * rates["input"] / 1_000
                        output_cost = fixed_output_tokens * rates["output"] / 1_000
                        cache_5m = model_tokens * rates["cached_5m"] / 1_000
                        cache_1h = model_tokens * rates["cached_1h"] / 1_000
                        cache_hit = model_tokens * rates["cache_hit"] / 1_000
                        total_cost = input_cost + output_cost
                        row[f"{model} (input)"] = input_cost
                        row[f"{model} (output, 4000)"] = output_cost
                        row[f"{model} (cache_5m)"] = cache_5m
                        row[f"{model} (cache_1h)"] = cache_1h
                        row[f"{model} (cache_hit)"] = cache_hit
                        row[f"{model} (total)"] = total_cost

                rows.append(row)

# Create dataframe
df = pd.DataFrame(rows)


df_filled = df.fillna(0)

# Create totals row
totals = {"Table Name": "TOTAL"}
for col in df_filled.columns:
    if col != "Table Name":  # sum all numeric columns
        totals[col] = df_filled[col].sum()

# Append totals row
df = pd.concat([df_filled, pd.DataFrame([totals])], ignore_index=True)


# Save to Excel
output_file = "token_costs_gpt5_claude.xlsx"
df.to_excel(output_file, index=False)

# Load workbook with openpyxl
wb = load_workbook(output_file)
ws = wb.active

# Loop over all columns
for i, col in enumerate(df.columns, start=1):  # Excel columns are 1-indexed
    # Hide if column name does NOT contain "total" and is not first two columns
    if i > 2 and "total" not in col.lower():
        ws.column_dimensions[ws.cell(row=1, column=i).column_letter].hidden = True

# Save workbook
wb.save(output_file)
print(f"Saved Excel with hidden columns to {output_file}")


Saved Excel with hidden columns to token_costs_gpt5_claude.xlsx
